# keras embedding을 이용한 문서 분류
### 영화리뷰 데이터 대상

## train set의 크기가 더 큰 데이터 셋, 그리고 한글

In [1]:
!pip install keras tensorflow konlpy tweepy==3.10.0

In [2]:
import csv

text = []
y = []
with open('movie_data_new.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [3]:
print('Num of samples: {}'.format(len(text)))
print('Movie titles of reivews: {}'.format(set(y)))

# total samples 이 14967 
# 예측하고자 하는 class 는 7개

Num of samples: 14967
Movie titles of reivews: {'인피니티 워', '범죄도시', '라라랜드', '코코', '택시운전사', '곤지암', '신과함께'}


In [4]:
label_dict = dict(zip(set(y),range(len(set(y)))))
y = [label_dict[c] for c in y]
print(len(y))
print(y[-10:])

14967
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [8]:
from keras.utils.np_utils import to_categorical
y = to_categorical(y)

In [9]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

# Java 설치가 도저히 안됨.......
# 하지만, 무엇을 하려하는지 정리를 열심히 하는 것으로 대체.

JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.

In [7]:
print(twitter_tag.nouns(text[4]))

# 명사 형태만! 

NameError: name 'twitter_tag' is not defined

In [ ]:
from seqGenKor import SequenceGenKor

In [ ]:
num_words = 10000
seq = SequenceGenKor(twitter_tag.nouns, num_words=num_words, min_token_len=1)
#seq = SequenceGenKor(twitter_tag.morphs, num_words=num_words, min_token_len=1)
seq.fit_on_texts(text)
X = seq.texts_to_sequences(text)
print('voca_size', len(seq.voca))

In [ ]:
print(X[:12])

In [ ]:
text[:12]

# str 데이터 자체를 명사 데이터에 대해서만 특정 정해진 숫자로 embedding 을 한다고 할 수 있음.

In [ ]:
from keras import preprocessing

maxlen = 20

X = preprocessing.sequence.pad_sequences(X, maxlen=maxlen, truncating='pre')
print(X[0])

# 한 sequence 당 등장할 수 있는 max를 20으로 결정하고 그보다 적으면 zero padding 처리 한다고 볼 수 있음. 
# 전처리를 통해 shape 자체를 맞추는 과정이라고 할 수 있음.

In [ ]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, categories를 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count:', len(X_train))
print('Test set count:', len(X_test))
print('Test samples:', y_test[:20])

# y label 은 one hot 형태로 전환.
# train 은 8 / test 는 2 의 비율로 데이터 셋 구축

In [ ]:
from keras.layers import SimpleRNN, LSTM
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(num_words, 32))
#model.add(SimpleRNN(32))
model.add(LSTM(32))
model.add(Dense(7, activation='softmax'))
model.summary()

# 위에서 전처리한 sequence 에 대해 rnn 에 보다 학습하기 좋은 상태로 만들기 위해 normalized 된 값의 범위와 shape 형태로 데이터 자체를 embedding
# 기본적인 RNN 모델에 해당하는 GRU 와 LSTM 중 LSTM 을 feature extraction으로 활용하여 모델링
# 마지막 classfier 는 fully connected layer 에 마지막 activation 으로 multi classification에 활용하는 softmax 을 활용.

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, 
                    epochs=30,
                    batch_size=256,
                    validation_split=0.2)

# loss function 은 데이터의 분포를 찾는 ce loss를 활용. classification task에 적합하다고 판단 됨.
# optimizer 는 rmsprop으로 활용. 무엇을 사용하든 큰 차이는 없을 것 같음. adam 이나 SGD도 고려해볼만 하다고 판단함.
# batch 는 빠른 수렴을 위해 하드웨어 스펙에 맞게 최대한 크게!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

history_dict = history.history
acc = history_dict['acc']
val_acc = history_dict['val_acc']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
score = model.evaluate(X_test, y_test)
print(score)

# 역시 fitting 일어남. 데이터가 적은 것도 한 몫 할 것 같음.
# 성능 향상을 위해 BiLSTM을 고려하는 건 어떤가 ..?
# 데이터 셋을 train : test = 9 : 1 로 하면 어떨까 ..? 딥러닝은 데이터가 훨씬 많이 필요하므로 학습 데이터를 더욱 강화하니까..?
